In [16]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from decouple import config
import time
import pandas as pd 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options



# Inicializar el driver de Selenium con las opciones definidas
driver = webdriver.Chrome()

# Ir a la página de inicio de LinkedIn
driver.get("https://www.bumeran.com.pe/login?returnTo=/")


user =config("USER_LKN")
clave =config("PASS_LKN")

# Encontrar el campo de inicio de sesión y enviar credenciales
username = driver.find_element(By.ID,"email")
username.send_keys(user)
password = driver.find_element(By.ID,"password")
password.send_keys(clave)
time.sleep(3)

boton1=driver.find_element(By.XPATH,'//a[text()=" Acepto"]')
boton1.click()


# Hacer clic en el botón de inicio de sesión
boton=driver.find_element(By.ID,"ingresar")
boton.click()

time.sleep(10)

# Inicializar la lista de datos
datas = []

for i in range(0, 5):

    # Construir la URL
    if i == 0:
        url = 'https://www.bumeran.com.pe/empleos-busqueda-data.html'
    else:
        url = f'https://www.bumeran.com.pe/empleos-busqueda-data.html?page={i+1}'
    
    # Navegar a la URL
    driver.get(url)
    time.sleep(5)

    # Encontrar la tabla en la página

    table = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.CLASS_NAME, 'sc-kdPYzB.egIfIy')))

    table_rows = table.find_elements(By.TAG_NAME, 'div') 
    print(f"Found {len(table_rows)} rows on the table from page {i}")

    tablex = []



    # Iterar a través de las filas de la tabla
    for row in table_rows:
        # Usar los selectores CSS correctos para su sitio web
        title_elements = row.find_elements(By.CLASS_NAME,"sc-fhiYOA.zGAkU") 
        company_elements = row.find_elements(By.CLASS_NAME,"sc-flvzOl.kJsfXU")
        location_elements = row.find_elements(By.CLASS_NAME,'sc-cRULEh.dNmbcO')
        link_elements = row.find_elements(By.CLASS_NAME, 'sc-jkPxnQ.ioakMb')

        title = title_elements[0].text if title_elements else None
        company = company_elements[0].text if company_elements else None
        location = location_elements[0].text if location_elements else None


        # Extraer el texto de los elementos encontrados
        title = title_elements[0].text if title_elements else None
        link = link_elements[0].get_attribute('href') if link_elements else None
        # Imprimir los detalles de cada fila
        print(f'Title: {title}, company: {company}, location: {location},Link: {link}')

        # Agregar los datos a la lista
        tablex.append({"title": title, "company": company, "location": location, "Link": link})

    # Convertir la lista en un DataFrame de pandas y agregarlo a la lista de datas
    df = pd.DataFrame(tablex)
    datas.append(df)

# Cerrar el driver de Selenium
#driver.quit()


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=117.0.5938.92)
Stacktrace:
	GetHandleVerifier [0x00007FF773867892+54818]
	(No symbol) [0x00007FF7737D6AC2]
	(No symbol) [0x00007FF77368DA3B]
	(No symbol) [0x00007FF77366DEE8]
	(No symbol) [0x00007FF7736F3237]
	(No symbol) [0x00007FF773706C6F]
	(No symbol) [0x00007FF7736EE883]
	(No symbol) [0x00007FF7736C3691]
	(No symbol) [0x00007FF7736C48D4]
	GetHandleVerifier [0x00007FF773BCB992+3610402]
	GetHandleVerifier [0x00007FF773C21860+3962352]
	GetHandleVerifier [0x00007FF773C19D4F+3930847]
	GetHandleVerifier [0x00007FF773903646+693206]
	(No symbol) [0x00007FF7737E1628]
	(No symbol) [0x00007FF7737DD934]
	(No symbol) [0x00007FF7737DDA62]
	(No symbol) [0x00007FF7737CE113]
	BaseThreadInitThunk [0x00007FFF2A8F257D+29]
	RtlUserThreadStart [0x00007FFF2C04AA68+40]


In [ ]:
final_df = pd.concat(datas, ignore_index=True)
final_df

,title,company,location,Link
0,Data Engineer,NTT DATA,"Chorrillos, Lima",https://www.bumeran.com.pe/empleos/data-engine...
1,Data Engineer,NTT DATA,"Chorrillos, Lima",None
2,None,None,None,None
3,Data Engineer,NTT DATA,None,None
4,None,NTT DATA,None,None
...,...,...,...,...
2448,None,None,None,None
2449,None,None,None,None
2450,None,None,None,None
2451,None,None,None,None


In [ ]:
final_df = final_df[final_df['Link'].str.len() > 5]


In [ ]:
# Asumiendo que final_df ya está creado
palabras_clave = [ 
    "Datos",
    "Analista", 
    "Big Data", 
    "Scientist",
    "Científico",
    "Azure", 
    "Analyst", 
    "Analista",
    "engineer",
    "ingeniero"
]


palabras_excluir = [
    "Administrador", 
    "Arquitecto", 
    "Subgerente",
    "AWS",
    "Financiero",
    "Verificador",
    "Auxiliar",
    "Practicante",
    "Jefe",
    "Coordinador",
    "Operador"
]


regex_incluir = '|'.join(palabras_clave)
regex_excluir = '|'.join(palabras_excluir)

final_df_filtrado = final_df[final_df['title'].str.contains(regex_incluir, case=False, na=False)]
final_df_filtrado = final_df_filtrado[~final_df_filtrado['title'].str.contains(regex_excluir, case=False, na=False)]


In [ ]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# Asumiendo que final_df_filtrado ya está creado

# Crea un nuevo libro de trabajo y una hoja de trabajo
wb = Workbook()
ws = wb.active

# Transfiere los datos del DataFrame a la hoja de trabajo, creando hiperlinks para la columna de links
for i, row in enumerate(dataframe_to_rows(final_df_filtrado, index=False, header=True)):
    row = list(row)
    if i == 0:
        ws.append(row)
    else:
        ws.append(row[:-1])
        ws['D%d' % (i+1)].hyperlink = row[-1]
        ws['D%d' % (i+1)].style = 'Hyperlink'

# Guarda el archivo
wb.save('Trabajo.xlsx')
